In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, Bidirectional, LSTM, Dense
from keras.regularizers import l1
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import numpy as np
import pickle
import tqdm as tqdm

In [2]:

max_words = 10000  # Maximum number of unique words to consider
max_sequence_length = 250  # Maximum length of each
embedding_dim = 300  # Dimension of the GloVe word embeddings
negation_words = ['not', 'no', 'never']  # Words to be considered as negation words
accumulation_steps = 4
learning_rates = [2e-5, 5e-5, 1e-4]
patience = 2  # Set the number of epochs to wait before stopping if validation loss doesn't decrease
# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coeffs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coeffs
    return embeddings_index


def create_embedding_matrix(embeddings_index, word_index, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


def sentiment_to_binary(sentiment):
    if sentiment == 'positive':
        return 1
    else:
        return 0


def preprocess_text_with_negation(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Replace any negation words with "not_" to preserve their meaning in the model
    for negation_word in negation_words:
        text = text.replace(negation_word, "not_" + negation_word)
    
    # Remove any non-alphanumeric characters
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    
    return text

In [3]:
df = pd.read_csv('../IMDB Dataset.csv')

# Apply the preprocess_text_with_negation function to the 'review' column
df['review'] = df['review'].apply(preprocess_text_with_negation)

df['sentiment'] = df['sentiment'].apply(sentiment_to_binary)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the text
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=250)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=250)

# Convert the tokenized data into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(1000).batch(4)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(4)

# Fine-tune the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

print('Training the model...')
model.fit(train_dataset, epochs=3, batch_size=4, validation_data=test_dataset)

# Evaluate the model
y_pred_logits = model.predict(test_dataset, batch_size=4)
y_pred = np.argmax(y_pred_logits.logits, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))



# optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# best_val_loss = float('inf')

# epochs_without_improvement = 0
# patience = 2

# for epoch in tqdm.tqdm(range(20), desc="Epochs", ncols=100):  # Increase the maximum number of epochs since we're using early stopping
#     epoch_loss = 0.0
#     epoch_steps = 0

#     accumulated_gradients = [tf.zeros_like(v) for v in model.trainable_variables]

#     for step, (x_batch, y_batch) in enumerate(train_dataset):
#         with tf.GradientTape() as tape:
#             logits = model(x_batch, training=True).logits
#             batch_loss = loss(y_batch, logits)

#         gradients = tape.gradient(batch_loss, model.trainable_variables)
#         accumulated_gradients = [(acc_grad + grad) for acc_grad, grad in zip(accumulated_gradients, gradients)]

#         if (step + 1) % accumulation_steps == 0:
#             optimizer.apply_gradients(zip(accumulated_gradients, model.trainable_variables))
#             accumulated_gradients = [tf.zeros_like(v) for v in model.trainable_variables]

#         epoch_loss += batch_loss.numpy()
#         epoch_steps += 1

#         if step % 500 == 0:
#             print(f"  Step {step}, loss: {batch_loss.numpy()}")

#     epoch_loss /= epoch_steps
#     print(f"  Epoch loss: {epoch_loss}")


#     # Calculate validation loss
#     val_loss = 0.0
#     val_steps = 0
#     for x_batch, y_batch in test_dataset:
#         logits = model(x_batch, training=False).logits
#         batch_loss = loss(y_batch, logits)
#         val_loss += batch_loss.numpy()
#         val_steps += 1
#     val_loss /= val_steps
#     print(f"  Validation loss: {val_loss}")

#     # Early stopping
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         epochs_without_improvement = 0
#         # Save the best model
#         model.save_pretrained('sentiment_analysis_bert_/')
#         tokenizer.save_pretrained('sentiment_analysis_bert_/')
#     else:
#         epochs_without_improvement += 1
#         if epochs_without_improvement >= patience:
#             print("Early stopping triggered. Stopping training.")
#             break

# Save the fine-tuned model
model.save_pretrained('sentiment_analysis_bert_/')
tokenizer.save_pretrained('sentiment_analysis_bert_/')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training the model...
Epoch 1/3
   18/10017 [..............................] - ETA: 26:19 - loss: 0.7160 - accuracy: 0.4444

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np

# Load the pre-trained model and tokenizer
model_path = 'sentiment_analysis_bert'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = TFBertForSequenceClassification.from_pretrained(model_path)

def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=512)

    # Perform the sentiment prediction
    outputs = model(inputs)
    logits = outputs.logits

    # Convert the logits to probabilities
    probabilities = tf.nn.softmax(logits, axis=-1).numpy()

    # Get the predicted class (0: Negative, 1: Positive)
    predicted_class = np.argmax(probabilities, axis=-1)

    return predicted_class, probabilities

# while loop to take input from user
while True:
    # Take input from user
    text = input("Enter a sentence to predict its sentiment (or 'q' to quit): ")
    
    # Check if the user wants to quit
    if text == 'q':
        break
    
    # Predict the sentiment
    predicted_class, probabilities = predict_sentiment(text)
    
    # Print the results
    print(text)
    print("Predicted sentiment: {}".format(predicted_class[0]))
    print("Negative probability: {}".format(probabilities[0][0]))
    print("Positive probability: {}".format(probabilities[0][1]))


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at sentiment_analysis_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Lets go
Predicted sentiment: 0
Negative probability: 0.6447935700416565
Positive probability: 0.3552064299583435
not good
Predicted sentiment: 0
Negative probability: 0.9978436231613159
Positive probability: 0.0021563174668699503
not bad
Predicted sentiment: 0
Negative probability: 0.9060649275779724
Positive probability: 0.0939350500702858
good
Predicted sentiment: 1
Negative probability: 0.012125497683882713
Positive probability: 0.987874448299408
bad
Predicted sentiment: 0
Negative probability: 0.999241828918457
Positive probability: 0.0007581330719403923
not bad
Predicted sentiment: 0
Negative probability: 0.9060649275779724
Positive probability: 0.0939350500702858
quit
Predicted sentiment: 0
Negative probability: 0.998991072177887
Positive probability: 0.0010089060524478555


In [2]:
from transformers import pipeline

# Load the sentiment analysis pipeline
nlp = pipeline("sentiment-analysis")

# Loop to take user input and get the sentiment analysis output
while True:
    user_input = input("Enter a sentence: ")
    if user_input.lower() == "quit":
        break
    else:
        result = nlp(user_input)
        print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.2f}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are 

Sentiment: POSITIVE, Score: 1.00
Sentiment: NEGATIVE, Score: 1.00
Sentiment: NEGATIVE, Score: 1.00
Sentiment: POSITIVE, Score: 0.75
